In [1]:
import numpy as np
from scipy.optimize import minimize
from datetime import date
from scipy.stats import norm

# 1번 문제

In [2]:
S = 273.55
K1 = 257.5
K2 = 260.0
r = 0.0155
T = date(2019, 10, 10)
t = date(2019, 9, 30)
C1 = 16.15
C2 = 14.00
time_to_expiration = (T - t)
time_to_expiration = time_to_expiration.days/365

In [3]:
def calcul_IV(x): # input : (IV, C, K)
    iv = x[0]
    C = x[1]
    K = x[2]
    d1 = (np.log(S / K) + (r + (iv ** 2) / 2) * time_to_expiration) / (iv * np.sqrt(time_to_expiration))
    d2 = d1 - (iv*np.sqrt(time_to_expiration))
    result = S*norm.cdf(d1) - K*np.exp((-1)*r*time_to_expiration)*norm.cdf(d2)
    return np.abs(result - C)

In [4]:
bnds1 = ((0, np.inf), (C1, C1), (K1, K1))
minimize(calcul_IV, [0.25, C1, K1], bounds = bnds1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


      fun: 0.009326100349461797
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.        , -1.00000008, -0.99957447])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 8
      nit: 1
   status: 0
  success: True
        x: array([  0.  ,  16.15, 257.5 ])

In [5]:
bnds2 = ((0, np.inf), (C2, C2), (K2, K2))
minimize(calcul_IV, [0.25, C2, K2], bounds = bnds2)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


      fun: 5.454808160720859e-09
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([5.3524559 , 1.00000008, 0.91881418])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 188
      nit: 6
   status: 0
  success: True
        x: array([2.1829291e-01, 1.4000000e+01, 2.6000000e+02])

In [6]:
calcul_IV([0, 16.1500000, 257.500000]) # IV : 0 >> 결과값이 0으로 수렴x (0.009326~ 으로 수렴.) 해가 없음.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


0.009326100349461797

In [7]:
calcul_IV([0.21829291, 14.0000000, 260.000000]) # IV : 0.21829291  >> 결과값이 0으로 수렴.

3.781025270654936e-09

### 그러므로 결과는 다음과 같다.
- 옵션 1의 Imvol은 구할 수 없다. >> 최적화 툴을 사용해도 할 수 없는 것을 보아 "해가 없는" 방정식이 나올 것 같다.
- 옵션 2의 Imvol은 구할 수 있다. >> IV이 0.21829291일 때, 블랙숄즈로 구한 콜옵션 가격이 시장가와 같게 된다.

옵션 1의 implied volatility(이하 IV)는 왜 안나오는 것일까? 일단 IV는 해당 시점에 옵션의 가치에 맞춰서 결정되는 블랙숄즈 방정식 기반의 변동성을 의미한다. 그렇다면 결국 IV는 수식에 의해서 결정된다는 것인데, 수식에 항상 실수인 단 한개의 근이 있을까? 라는 생각을 할 수 있게 된다. 어쩌면 여러개의 근이 있을 수 있고 어쩌면 근이 없거나 복소수인 근이 있을 수도 있는 것이 방정식이다. 검토를 해본 결과, 후자인 경우가 이 경우라고 보여진다.

# 2번 문제

In [74]:
def Thomas_algo1(a, b, c, d):
    n = len(d) 
    _c = np.zeros(6)
    _d = np.zeros(6)
    x = np.zeros(6)
    _c[0] = c[0]/a[0]
    _d[0] = d[0]/a[0]
    for i in range(1, n):
        _c[i] = ( c[i] / (a[i]-b[i]*_c[i-1]) )
        _d[i] = ( (d[i] - b[i] * _d[i-1]) / (a[i] - b[i] * _c[i-1]) )
        
    x[n-1] = _d[n-1]
    for i in range(n-2, -1, -1):
        x[i] = _d[i] - _c[i] * x[i+1]
    
    return x

In [75]:
def Thomas_algo2(a, b, c, d):
    n = len(d) 
    _a = np.zeros(6)
    _d = np.zeros(6)
    x = np.zeros(6)
    _a[0] = a[0]
    _d[0] = d[0]
    for i in range(1, n):
        _a[i] = a[i] - b[i] * c[i-1]/_a[i-1]
        _d[i] =  d[i] - b[i] * _d[i-1] / _a[i-1]
    
    _a[n-1] = a[n-1] - b[n-1]*c[n-2]/_a[n-2]
    _d[n-1] = d[n-1] - b[n-1]*_d[n-2]/_a[n-2]
        
    x[n-1] = _d[n-1]/_a[n-1]
    for i in range(n-2, -1, -1):
        x[i] = _d[i]/_a[i] - c[i]/_a[i] * x[i+1]
    return x

In [76]:
def extract_var(mat): # mat : tridiagonal matrix
    n = len(mat)
    a = np.zeros(6)
    a[0] = (mat[0][0])
    b = np.zeros(6)
    c = np.zeros(6)
    c[0] = (mat[0][1])
    for i in range(1, n-1):
        a[i] = (mat[i][i])
        b[i] = (mat[i][i-1])
        c[i] = (mat[i][i+1])
    a[n-1] = (mat[n-1][n-1])
    b[n-1] = (mat[n-1][n-2])
    return a, b, c

In [77]:
_input = np.array([[1,2,0,0,0,0],[-2,1,1,0,0,0],[0,.5,1,.5,0,0],[0,0,-2,1,.5,0],[0,0,0,1,.2,.3],[0,0,0,0,1,1]],dtype=float)
_d = np.array([2, -1, 0.5, 2.3, -5, 0.25])

In [78]:
_a, _b, _c = extract_var(_input)

In [79]:
def Thomas_algorithm(A, _d):
    _a, _b, _c = extract_var(A)
    return Thomas_algo1(_a, _b, _c, _d)

In [80]:
Thomas_algorithm(_input, _d)

array([  1.59609375,   0.20195312,   1.99023438,  -3.18242188,
        18.92578125, -18.67578125])

#### 그러므로, x는 [  1.59609375,   0.20195312,   1.99023438,  -3.18242188, 18.92578125, -18.67578125]